In [1]:
import pandas as pd
import numpy as np
import itertools, re
from nltk.stem import StemmerI
from nltk.metrics import edit_distance

In [2]:
class IndianNameStemmer(StemmerI):
    def stem(self, token):
        newtup=list()
        for i in token:
            i = re.sub(r'[^a-zA-Z]', '', i)
            i = re.sub(r'(\w)\1+',r'\1', i)
            i = i[:-2] if i.endswith(('bi', 'ji', 'so')) else i            
            i = i[:-3] if i.endswith(('bai', 'ben', 'beg', 'tai', 'sab', 'rao', 'rav', 'kha', 'lal', 'dev', 'deo')) else i
            i = i[:-4] if i.endswith(('bhai', 'bhau', 'sing', 'devi')) else i
            i = i[:-5] if i.endswith(('saheb', 'singh' , 'shing', 'kumar')) else i
            for r in (("tha", "ta"), ("gi", "ji"), ("bh", "b"), ("v", "w"), ("yu", "u"), ("gh", "g"), ("dh", "d"), ("sh", "s"), 
                      ("bh","b"), ("ph","f"), ("z","j"), ("gaw", "gaon"), ("gav", "gaon"), ("g","j"), ("i", "e"), ("a","") ):
                i = i.replace(*r)

            newtup.append(''.join(i for i, _ in itertools.groupby(i)))
        return tuple(newtup)


In [3]:
s = IndianNameStemmer()
s.stem(['aanaso/shantanu' , 'asha', 'bagirang' , 'bajirang'])

('nsosntnu', 's', 'bjernj', 'bjernj')

In [4]:
df=pd.read_excel('correct.xlsx')
df.columns=['wrong', 'correct']

In [5]:
df['updated']=df['wrong'].astype(str).apply(lambda x: x.split('/')).apply(s.stem)
df['updated1']=df['correct'].astype(str).apply(lambda x: x.split('/')).apply(s.stem)

In [6]:
ndf=df[df['updated'] != df['updated1']]

In [7]:
mylist=list()
mylist1=list()
for i, rows in ndf.iterrows():
    mylist.append(edit_distance(rows['updated'][0], rows['updated1'][0], substitution_cost=2))
    mylist1.append(edit_distance(rows['wrong'], rows['correct'], substitution_cost=2))


In [8]:
ndf['edu1'] = mylist
ndf['wc'] = mylist1

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [9]:
ndf=ndf[~np.logical_or((ndf['wc'] == 1) , (ndf['edu1'] == 1))]

In [10]:
ndf['ulen'] = ndf['updated'].str[0].apply(len)
ndf = ndf.sort_values('ulen')

In [11]:
ndf.to_excel('to_study1.xlsx')

In [12]:
ndf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92954 entries, 17562 to 127777
Data columns (total 7 columns):
wrong       92954 non-null object
correct     92954 non-null object
updated     92954 non-null object
updated1    92954 non-null object
edu1        92954 non-null int64
wc          92954 non-null int64
ulen        92954 non-null int64
dtypes: int64(3), object(4)
memory usage: 5.7+ MB


# compare names from 2 dataframes

In [13]:
from io import StringIO

myst="""shantanu prabhakar oka, 905034 , 19:44   
sammeer annashaaheb goankar, 905094  , 19:33
paravatibai vittal shelke,  905154 ,   21:56
"""
u_cols=['name', 'my_index', 'current_tm']

myf = StringIO(myst)
import pandas as pd
df = pd.read_csv(StringIO(myst), sep=',', names = u_cols)

In [14]:
myst="""shantanu34 prabakar oak, 905034 , 19:44   
sammir anashaheb goankar, 905094  , 19:33
parwati vithal bholke,  905154 ,   21:56
samir anashaheb gavkar, 905094  , 19:33

"""
u_cols=['name', 'my_index', 'current_tm']

myf = StringIO(myst)
import pandas as pd
df1 = pd.read_csv(StringIO(myst), sep=',', names = u_cols)

In [15]:
df['updated']=df['name'].astype(str).apply(lambda x: x.split()).apply(s.stem)
df1['updated1']=df1['name'].astype(str).apply(lambda x: x.split()).apply(s.stem)

In [16]:
df1

,name,my_index,current_tm,updated1
0,shantanu34 prabakar oak,905034,19:44,"(sntnu, prbkr, ok)"
1,sammir anashaheb goankar,905094,19:33,"(smer, nsheb, jonkr)"
2,parwati vithal bholke,905154,21:56,"(prwte, wetl, bolke)"
3,samir anashaheb gavkar,905094,19:33,"(smer, nsheb, jonkr)"


In [17]:
df

,name,my_index,current_tm,updated
0,shantanu prabhakar oka,905034,19:44,"(sntnu, prbkr, ok)"
1,sammeer annashaaheb goankar,905094,19:33,"(smer, nsheb, jonkr)"
2,paravatibai vittal shelke,905154,21:56,"(prwte, wetl, selke)"


In [18]:
df.merge(df1, how='outer', left_on=['updated'], right_on=['updated1'], indicator=True)

,name_x,my_index_x,current_tm_x,updated,name_y,my_index_y,current_tm_y,updated1,_merge
0,shantanu prabhakar oka,905034.0,19:44,"(sntnu, prbkr, ok)",shantanu34 prabakar oak,905034.0,19:44,"(sntnu, prbkr, ok)",both
1,sammeer annashaaheb goankar,905094.0,19:33,"(smer, nsheb, jonkr)",sammir anashaheb goankar,905094.0,19:33,"(smer, nsheb, jonkr)",both
2,sammeer annashaaheb goankar,905094.0,19:33,"(smer, nsheb, jonkr)",samir anashaheb gavkar,905094.0,19:33,"(smer, nsheb, jonkr)",both
3,paravatibai vittal shelke,905154.0,21:56,"(prwte, wetl, selke)",NaN,NaN,NaN,NaN,left_only
4,NaN,NaN,NaN,NaN,parwati vithal bholke,905154.0,21:56,"(prwte, wetl, bolke)",right_only


In [19]:
mylist=list()
for i, rows in df.iterrows():
        for i1, rows1 in df1.iterrows():
            mylist.append(edit_distance(rows['updated'], rows1['updated1'], substitution_cost=2))

In [20]:
fdf = pd.merge(df.assign(key=0), df1.assign(key=0), on='key').drop('key', axis=1)

In [21]:
fdf['eud'] = mylist

In [22]:
fdf

,name_x,my_index_x,current_tm_x,updated,name_y,my_index_y,current_tm_y,updated1,eud
0,shantanu prabhakar oka,905034,19:44,"(sntnu, prbkr, ok)",shantanu34 prabakar oak,905034,19:44,"(sntnu, prbkr, ok)",0
1,shantanu prabhakar oka,905034,19:44,"(sntnu, prbkr, ok)",sammir anashaheb goankar,905094,19:33,"(smer, nsheb, jonkr)",6
2,shantanu prabhakar oka,905034,19:44,"(sntnu, prbkr, ok)",parwati vithal bholke,905154,21:56,"(prwte, wetl, bolke)",6
3,shantanu prabhakar oka,905034,19:44,"(sntnu, prbkr, ok)",samir anashaheb gavkar,905094,19:33,"(smer, nsheb, jonkr)",6
4,sammeer annashaaheb goankar,905094,19:33,"(smer, nsheb, jonkr)",shantanu34 prabakar oak,905034,19:44,"(sntnu, prbkr, ok)",6
5,sammeer annashaaheb goankar,905094,19:33,"(smer, nsheb, jonkr)",sammir anashaheb goankar,905094,19:33,"(smer, nsheb, jonkr)",0
6,sammeer annashaaheb goankar,905094,19:33,"(smer, nsheb, jonkr)",parwati vithal bholke,905154,21:56,"(prwte, wetl, bolke)",6
7,sammeer annashaaheb goankar,905094,19:33,"(smer, nsheb, jonkr)",samir anashaheb gavkar,905094,19:33,"(smer, nsheb, jonkr)",0
8,paravatibai vittal shelke,905154,21:56,"(prwte, wetl, selke)",shantanu34 prabakar oak,905034,19:44,"(sntnu, prbkr, ok)",6
9,paravatibai vittal shelke,905154,21:56,"(prwte, wetl, selke)",sammir anashaheb goankar,905094,19:33,"(smer, nsheb, jonkr)",6


In [23]:
gdf=fdf.groupby(['name_x'])['eud'].min()

In [24]:
gdf=gdf.reset_index()

In [25]:
gdf

,name_x,eud
0,paravatibai vittal shelke,2
1,sammeer annashaaheb goankar,0
2,shantanu prabhakar oka,0


In [26]:
gdf.merge(fdf, how='left', left_on=['name_x', 'eud'], right_on=['name_x', 'eud'])

,name_x,eud,my_index_x,current_tm_x,updated,name_y,my_index_y,current_tm_y,updated1
0,paravatibai vittal shelke,2,905154,21:56,"(prwte, wetl, selke)",parwati vithal bholke,905154,21:56,"(prwte, wetl, bolke)"
1,sammeer annashaaheb goankar,0,905094,19:33,"(smer, nsheb, jonkr)",sammir anashaheb goankar,905094,19:33,"(smer, nsheb, jonkr)"
2,sammeer annashaaheb goankar,0,905094,19:33,"(smer, nsheb, jonkr)",samir anashaheb gavkar,905094,19:33,"(smer, nsheb, jonkr)"
3,shantanu prabhakar oka,0,905034,19:44,"(sntnu, prbkr, ok)",shantanu34 prabakar oak,905034,19:44,"(sntnu, prbkr, ok)"
